### Suraj Desai:  Dataset Preprocessing

In [6]:
import pandas as pd
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [7]:
data = pd.read_excel("dataset/TrainData.xlsx")

In [8]:
data

,sentence,query
0,Find the total revenue of products sold in D...,"SELECT City, SUM(ValueofSalesUnit * ofSalesUn..."
1,List all products sold with their quantities...,"SELECT ProductName, SUM(ofSalesUnitSold) AS T..."
2,Get the names of customers who made pre-orde...,"SELECT FirstName, LastName FROM Customers INN..."
3,Find the product that generated the highest ...,"SELECT ProductName, SUM(ValueofSalesUnit * of..."
4,List the top 5 products with the highest sal...,"SELECT ProductName, SUM(ofSalesUnitSold) AS T..."
...,...,...
95,Find the total quantity of each product sold...,"SELECT ProductName, SalesUnit, SUM(ofSalesUn..."
96,Get the average price of products sold in ea...,"SELECT City, AVG(Menu_Items.Price) AS Averag..."
97,List all products that were sold at least on...,SELECT DISTINCT ProductName FROM Orders_Dece...
98,Find the least popular product based on the ...,"SELECT ProductName, SUM(ofSalesUnitSold) AS ..."


In [9]:
prepared_data = []

#### Created function to format train data as per the format mentioned in docs

In [10]:
def prepare_data(row):
    sent = row[0]
    query = row[1]
    
    datapoint = {"messages": 
                 [
                    {"role": "system", "content": "Jaarus is SQL Parser who learns scghema and generates mysql query"},
                    {"role": "user", "content": sent},
                    {"role": "assistant", "content": query}
                 ]
                }
    
    prepared_data.append(datapoint)
    

In [11]:
data[["sentence", "query"]].apply(prepare_data, axis=1 )

0     None
1     None
2     None
3     None
4     None
      ... 
95    None
96    None
97    None
98    None
99    None
Length: 100, dtype: object

In [12]:
import json

f = open('dataset/schema.json')

schema = json.load(f)


f.close()


In [13]:
schema

[{'messages': [{'role': 'system',
    'content': 'Jaarus is SQL Parser who learns scghema and generates mysql query'},
   {'role': 'user',
    'content': "Here is the schema: CREATE TABLE 'Sales_Unit' ('ID' int NOT NULL AUTO_INCREMENT, 'ProductName' text,'SalesUnitVal' int DEFAULT NULL, PRIMARY KEY ('ID'), UNIQUE KEY 'ID_UNIQUE' ('ID') ) ENGINE=InnoDB AUTO_INCREMENT=4 DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci;"},
   {'role': 'assistant', 'content': 'Thanks for the schema'}]},
 {'messages': [{'role': 'system',
    'content': 'Jaarus is SQL Parser who learns scghema and generates mysql query'},
   {'role': 'user',
    'content': 'Here is the schema: CREATE TABLE `Products_Table` (`ID` int NOT NULL, `CategoryID` int DEFAULT NULL, `ProductName` text, `Description` text, PRIMARY KEY (`ID`), UNIQUE KEY `ID_UNIQUE` (`ID`), KEY `CategoryID_idx` (`CategoryID`), CONSTRAINT `CategoryID` FOREIGN KEY (`CategoryID`) REFERENCES `Categories` (`ID`)) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLA

In [14]:
len(schema)

8

In [106]:
file_path = "dataset/train_spider_and_others.json"

spider_data = []

with open(file_path) as f:
    for row in json.load(f):
        spider_data.append([row["question"], row["query"]])

In [110]:
spider_df = pd.DataFrame(data = spider_data, columns=["sentence", "query"])

In [111]:
spider_df.head()

,sentence,query
0,How many heads of the departments are older th...,SELECT count(*) FROM head WHERE age > 56
1,"List the name, born state and age of the heads...","SELECT name , born_state , age FROM head ORD..."
2,"List the creation year, name and budget of eac...","SELECT creation , name , budget_in_billions ..."
3,What are the maximum and minimum budget of the...,"SELECT max(budget_in_billions) , min(budget_i..."
4,What is the average number of employees of the...,SELECT avg(num_employees) FROM department WHER...


In [115]:
len(prepared_data)

100

In [116]:
spider_df[["sentence", "query"]].apply(prepare_data, axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
8654    None
8655    None
8656    None
8657    None
8658    None
Length: 8659, dtype: object

In [118]:
len(prepared_data)

8759

In [119]:
train_data = schema + prepared_data

In [120]:
file_path = "dataset/train_data_extended.jsonl"

with open(file_path, 'w') as file:
    for row in train_data:
        file.write(json.dumps(row) + '\n')

In [16]:
file_path = "dataset/train_data_schema_only.jsonl"

with open(file_path, 'w') as file:
    for row in schema:
        file.write(json.dumps(row) + '\n')